Step 1 - Create the Fact Table

In [9]:
/*Create the  Table 
[dbo].[FactCallsHandled] 
*/
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[FactCallsHandled]') AND type IN (N'U'))
DROP TABLE [dbo].[FactCallsHandled]
GO

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

CREATE TABLE [dbo].[FactCallsHandled](
	[ID] [INT] NOT NULL,
	[CallDate] [DATETIME] NULL,
	[AgentId] [INT] NULL,
	[AnsweredCall] [INT] NOT NULL,
	[MissedCall] [INT] NOT NULL,
	[FowardedCall] [INT] NOT NULL,
	[AdjustedDate]  [DATETIME] NULL,
) ON [PRIMARY]
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.137

Step 2 - CreateAgentDim

In [10]:
/*
Create Table [dbo].[DimAgent]  
*/
IF EXISTS
(
    SELECT *
    FROM sys.objects
    WHERE object_id = OBJECT_ID(N'[dbo].[DimAgent]')
          AND type IN ( N'U' )
)
    DROP TABLE [dbo].[DimAgent];
GO

SET ANSI_NULLS ON;
GO

SET QUOTED_IDENTIFIER ON;
GO

CREATE TABLE [dbo].[DimAgent]
(
    [ID] [INT] IDENTITY(1, 1) NOT NULL,
    [AgentName] [NVARCHAR](10) NOT NULL
) ON [PRIMARY];
GO


INSERT INTO [dbo].[DimAgent]
(
    [AgentName]
)
VALUES
('YasserM'),
('SusanP'),
('ThomasR');
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(3 rows affected)

Total execution time: 00:00:00.129

Step 3 - Generate\_Fact\_Table

In [11]:
/*============================================================
1) Delete all the records in the  fact table 
2) Drop any temp tables references in the script. 
 The reason for this is that so the scripts can be run multiple times. So can be 
 used training purposes.  Also that anyone use the same process shown in the
 session.
=============================================================*/
TRUNCATE TABLE [dbo].[FactCallsHandled];
DROP TABLE IF EXISTS #Names;
DROP TABLE IF EXISTS #AgentPerformance;

/*============================================================
create a temp table which contains names of the agents
=============================================================*/
CREATE TABLE #Names
(
    AgentID INT
);
/*============================================================
create the temp fact table which will contain the stats to be used
=============================================================*/
CREATE TABLE #AgentPerformance
(
    ID INT IDENTITY(1, 1),
    AgentID INT,
    DateUpdated DATETIME,
    CallsAnswered INT,
    CallsFowarded INT,
    CallMissed INT
);

INSERT INTO #Names
(
    AgentID
)
VALUES
(1),
(2),
(3);

/*============================================================
https://stackoverflow.com/questions/11141507/how-to-generate-a-range-of-dates-in-sql-server#11141660
=============================================================*/

DECLARE @start DATETIME,
        @end DATETIME;

/*get start date 6 months in the past
based on date when the script is ran*/
SET @start = CAST(DATEADD(MONTH, -6, DATEADD(dd, -1, GETDATE())) AS DATE);
/*get end date 6 days in the past
based on date when the script is ran*/
SET @end = CAST(DATEADD(DAY, -6, GETDATE()) AS DATE);

SELECT @start AS [Start],
       @end AS [End];
WITH n
AS (SELECT TOP (DATEDIFF(DAY, @start, @end) + 1)
           n = ROW_NUMBER() OVER (ORDER BY [object_id])
    FROM sys.all_objects),
     UserNames
AS (SELECT Na.AgentID,
           DATEADD(DAY, n - 1, @start) [Date]
    FROM n
        CROSS JOIN #Names AS Na)
INSERT INTO #AgentPerformance
(
    AgentID,
    DateUpdated
)
SELECT UserNames.AgentID,
       UserNames.Date
FROM UserNames;

DECLARE @AgentPerformanceRowCount AS INT;

SET @AgentPerformanceRowCount =
(
    SELECT COUNT(*)FROM #AgentPerformance
);
/*
https://www.sqlshack.com/how-to-generate-random-sql-server-test-data-using-t-sql/
*/
;
WITH randowvalues
AS (SELECT 1 id,
           CAST(RAND(CHECKSUM(NEWID())) * 100 AS INT) AS CallsAnswered,
           CAST(RAND(CHECKSUM(NEWID())) * 100 AS INT) AS CallsFowarded,
           CAST(RAND(CHECKSUM(NEWID())) * 100 AS INT) AS CallMissed
    UNION ALL
    SELECT id + 1,
           CAST(RAND(CHECKSUM(NEWID())) * 100 AS INT) AS CallsAnswered,
           CAST(RAND(CHECKSUM(NEWID())) * 100 AS INT) AS CallsFowarded,
           CAST(RAND(CHECKSUM(NEWID())) * 100 AS INT) AS CallMissed
    FROM randowvalues
    WHERE
        --id < 1000
        id < @AgentPerformanceRowCount)
INSERT INTO [dbo].[FactCallsHandled]
(
    ID,
    [AgentId],
    [CallDate],
    [AnsweredCall],
    [MissedCall],
    [FowardedCall]
)
SELECT RV.id,
       AP.AgentID,
       AP.DateUpdated,
       RV.CallsAnswered,
       RV.CallMissed,
       RV.CallsFowarded
FROM randowvalues AS RV
    INNER JOIN #AgentPerformance AS AP
        ON AP.ID = RV.id
OPTION (MAXRECURSION 0);

DROP TABLE IF EXISTS #Names;
DROP TABLE IF EXISTS #AgentPerformance;

(3 rows affected)

(1 row affected)

(540 rows affected)

(540 rows affected)

Total execution time: 00:00:00.078

Start,End
2022-07-14 00:00:00.000,2023-01-09 00:00:00.000


Lets look at whats in the tables

In the DimAgents Table

In [12]:
SELECT [ID]
      ,[AgentName]
  FROM [dbo].[DimAgent]

(3 rows affected)

Total execution time: 00:00:00.043

ID,AgentName
1,YasserM
2,SusanP
3,ThomasR


Lets look at whats in the tables

In the FactCallsHandled Table

In [13]:
SELECT [ID]
      ,[CallDate]
      ,[AgentId]
      ,[AnsweredCall]
      ,[MissedCall]
      ,[FowardedCall]
      ,[AdjustedDate]
  FROM [dbo].[FactCallsHandled]

(540 rows affected)

Total execution time: 00:00:00.054

ID,CallDate,AgentId,AnsweredCall,MissedCall,FowardedCall,AdjustedDate
1,2022-07-14 00:00:00.000,1,89,77,44,NULL
2,2022-07-15 00:00:00.000,1,24,47,99,NULL
3,2022-07-16 00:00:00.000,1,99,37,8,NULL
4,2022-07-17 00:00:00.000,1,4,48,34,NULL
5,2022-07-18 00:00:00.000,1,91,14,34,NULL
6,2022-07-19 00:00:00.000,1,29,19,3,NULL
7,2022-07-20 00:00:00.000,1,48,40,29,NULL
8,2022-07-21 00:00:00.000,1,32,53,45,NULL
9,2022-07-22 00:00:00.000,1,12,36,96,NULL
10,2022-07-23 00:00:00.000,1,39,21,52,NULL
